In [8]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import os
from re import search

In [99]:
def get_subdirectories(folder_path):
    subdirectories = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subdirectories.append(item)
    return subdirectories

def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files

def get_num(text):
    if isinstance(text, list):
        numbers = []
        for item in text:
            number_str = search(r'\d+(?=[^\d]*\.csv)', str(item))[0]
            if number_str:
                numbers.append(number_str)
        return numbers
    else:
        number_str = search(r'\d+(?=[^\d]*\.csv)', str(text))[0]
        return number_str if number_str else None

# def read_csv_to_tensor(file_path, tensor_name):
#     # Read CSV file as a pandas DataFrame
#     dataframe = pd.read_csv(file_path)
#     # Convert DataFrame to a tensor
#     tensor = torch.tensor(dataframe.values)
#     # Assign the tensor to the specified variable name
#     locals()[tensor_name] = tensor

def read_csv_to_tensor(file_path, tensor_name, tensors_dict):
    # Read CSV file as a pandas DataFrame
    dataframe = pd.read_csv(file_path)

    # Convert DataFrame to a tensor
    tensor = torch.tensor(dataframe.values)

    # Store the tensor in the dictionary with the specified variable name
    tensors_dict[tensor_name] = tensor

def read_csv_to_tensor2(file_path):#, tensor_name, tensors_dict):
    # Read CSV file as a pandas DataFrame
    return pd.read_csv(file_path)
    
    # Convert DataFrame to a tensor
    #tensor = torch.tensor(dataframe.values)

    # Store the tensor in the dictionary with the specified variable name
    #tensors_dict[tensor_name] = tensor


# main_df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
# index_label_df = pd.DataFrame({'index': [0, 2], 'label': ['Label1', 'Label2']})

# merged_df = pd.merge(main_df, index_label_df, left_index=True, right_on='index', how='left')
# final_df = merged_df.drop(['index', 'label'], axis=1).set_index('label')


folder_path = os.getcwd()+'/data/'
print(folder_path)
subdirectories = get_subdirectories(folder_path)
print(subdirectories)

file_extension = ['.csv','.csv.stepMixed']
label = ['data','labels']
tensors = {}

for i in subdirectories:
    folder_i = folder_path+i
    files_t = get_files_in_subdirectories(folder_i)
    files_n = list(dict.fromkeys(get_num(files_t)))
    for index_j,j in enumerate(files_n):
        name_variable = i+'_'+str(index_j)
        for index_k,k in enumerate(file_extension):
            files = get_files_in_subdirectories(folder_i,file_extension=k,file_contains=j)
            name_variableR = i+'_'+j+'_'+label[index_k]
            # print(f'variable: {name_variableR} person: {i} file_extension: {k} file_path: {files}')
            if index_k == 0:
                main_df = read_csv_to_tensor2(files[0])#,name_variable,tensors)
            else:
                label_df = pd.read_csv(files[0],names = ['start','end'])
                index_start = label_df.start.tolist()
                index_end = label_df.end.tolist()
        # Add a new column with a specific value for the matching row indexes
        main_df['label1'] = main_df.apply(lambda row: 'start' if row.name in index_start else '', axis=1)
        main_df['label2'] = main_df.apply(lambda row: 'end' if row.name in index_end else '', axis=1)
        # Define the condition for the new column
        condition1 = main_df['label1'] == 'start'
        condition2 = main_df['label2'] == 'end'
        # Assign values based on the condition using np.where()
        main_df['label'] = np.where(condition1,1,
                                np.where(condition2,2,0))
        main_df = main_df.drop(['label1', 'label2'], axis=1)
        # print(main_df)
        # Convert DataFrame to a tensor
        tensor = torch.tensor(main_df.values)
        # Store the tensor in the dictionary with the specified variable name
        tensors[name_variable] = tensor

# Labels are:
# 0 - Neither start nor end
# 1 - Start
# 2 - End

/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/
['person_8', 'person_7', 'person_12', 'person_5', 'person_2', 'person_3', 'person_4', 'person_11', 'person_10']


In [102]:
tensors['person_10_0']

tensor([[ 0.0000, -2.6150,  5.3916,  ...,  0.0148,  0.0101,  0.0000],
        [ 0.0000, -2.5290,  5.3612,  ...,  0.0143,  0.0200,  0.0000],
        [ 0.0000, -2.5290,  5.3612,  ...,  0.0143,  0.0200,  0.0000],
        ...,
        [ 0.0000, -0.4884,  5.8274,  ..., -0.3436, -0.2903,  0.0000],
        [ 0.0000, -0.5159,  5.8757,  ..., -0.3344, -0.2963,  0.0000],
        [ 0.0000, -0.5594,  5.8534,  ..., -0.3134, -0.3382,  0.0000]],
       dtype=torch.float64)